# 00 ? Shared Environment Setup

Run this notebook once at the start of each Colab session. It installs the lightweight thesis environment, finds the repo root, and verifies CUDA access plus dataset downloads.


## Install dependencies (Colab)
Skip if you already created the `peft-thesis` conda env locally.


In [5]:
%cd /content
!git clone https://github.com/mounikakarasu/thesis-peft-llm.git
%cd thesis-peft-llm

/content
Cloning into 'thesis-peft-llm'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 31 (delta 7), reused 24 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 15.30 KiB | 15.30 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/thesis-peft-llm


In [6]:
!pip install -q -U transformers datasets evaluate accelerate peft matplotlib pandas scikit-learn


## Locate repository and seed everything


In [7]:
import os, sys
os.chdir("/content/thesis-peft-llm")
if "/content/thesis-peft-llm" not in sys.path:
    sys.path.append("/content/thesis-peft-llm")
print("Current dir:", os.getcwd())

Current dir: /content/thesis-peft-llm


In [8]:
import os
import sys
from pathlib import Path

def find_project_root() -> Path:
    current = Path.cwd().resolve()
    for path in [current, *current.parents]:
        if (path / "src").exists() and (path / "notebooks").exists():
            return path
    raise RuntimeError("Unable to locate the repository root. Please run this notebook from inside the project.")

PROJECT_ROOT = find_project_root()
os.chdir(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
print(f"Project root: {PROJECT_ROOT}")


Project root: /content/thesis-peft-llm


In [9]:

from src.utils import GLOBAL_CONFIG, configure_tf32, get_device, set_seed_everywhere

set_seed_everywhere(GLOBAL_CONFIG.seed)
configure_tf32(GLOBAL_CONFIG.tf32)
device = get_device()
print(f"Seed: {GLOBAL_CONFIG.seed}")
print(f"Device: {device}")
if device.type == "cuda":
    import torch

    print(torch.cuda.get_device_name(0))


Seed: 42
Device: cuda
Tesla T4


/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


## Pre-download SST-2


In [10]:

from src import data

sst2 = data.load_sst2()
print(sst2)
print({split: len(ds) for split, ds in sst2.items()})


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sst2/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

sst2/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

sst2/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})
{'train': 67349, 'validation': 872, 'test': 1821}
